## Welcome to the ebook2audiobook free kaggle!
## 🌟 Features

- 📖 Converts eBooks to text format with Calibre.
- 📚 Splits eBook into chapters for organized audio.
- 🎙️ High-quality text-to-speech with [Coqui XTTS](https://huggingface.co/coqui/XTTS-v2), [Fairseq](https://github.com/facebookresearch/fairseq), [Bark](https://github.com/suno-ai/bark), [Vits](https://github.com/jaywalnut310/vits), and [Yourtts](https://github.com/Edresson/YourTTS).
- 🗣️ Optional voice cloning with your own voice file or choose from our pre-made fine-tuned models!
- 🌍 Supports multiple languages! Arabic (ar), Chinese (zh-cn), Czech (cs), Dutch (nl), English (en), French (fr), German (de), Hindi (hi), Hungarian (hu), Italian (it), Japanese (ja), Korean (ko), Polish (pl), Portuguese (pt), Russian (ru), Spanish (es), Turkish (tr), Vietnamese (vi), [+ 1107 languages via Fairseq](https://dl.fbaipublicfiles.com/mms/tts/all-tts-languages.html)
## Want to run locally for free? ⬇
## [Check out the ebook2audiobook github!](https://github.com/DrewThomasson/ebook2audiobook)

In [ ]:
# Update system packages
!sudo apt-get update -y

# Install prerequisites
!sudo apt-get install -y software-properties-common

# Add deadsnakes PPA for newer Python versions
!sudo add-apt-repository -y ppa:deadsnakes/ppa
!sudo apt-get update -y

# Install Python 3.11 and essentials
!sudo apt-get install -y python3.11 python3.11-distutils python3.11-venv

# Install pip for Python 3.11
!wget -q https://bootstrap.pypa.io/get-pip.py
!python3.11 get-pip.py

# Register Python 3.11 as an alternative
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 2

# Automatically set Python 3.11 as default (no prompt)
!sudo update-alternatives --set python3 /usr/bin/python3.11

# Verify Python version
!python3 --version

# Install common packages using pip for Python 3.11
!python3 -m pip install --upgrade pip
!python3 -m pip install numpy pandas matplotlib

In [ ]:
!python --version

In [ ]:
# @title 🚀 Run ebook2audiobook!

import os
import re
import subprocess
import time  # Import the time module
from IPython.display import HTML, display
import pkg_resources  # For checking installed packages

# Emojis for logs
CHECK_MARK = "✅"
CROSS_MARK = "❌"

def display_loading_bar(total_steps):
    """Displays a more visual text-based loading indicator in Colab."""
    print("\n---  LOADING...  Total steps:", total_steps, " ---") # Separator and loading start

def update_progress(step, total_steps):
  """Updates the text-based loading indicator with a bar-like style."""
  bar_length = 20  # Length of the "bar"
  progress_percent = int((step / total_steps) * 100)
  progress_filled = int(bar_length * step / total_steps)
  bar = '=' * progress_filled + '>' + ' ' * (bar_length - progress_filled -1 if (bar_length - progress_filled -1) >= 0 else 0) # Handle potential index error
  progress_text = f"[{bar}] {progress_percent}% ({step}/{total_steps})"
  print(f"---  PROGRESS:  {progress_text} ---") # Separator for progress update

def run_command_with_log(command, description, step_progress, total_step_commands):
    """Runs a command and logs its progress and outcome with emojis and duration."""
    print(f"\n{step_progress}/{total_step_commands}: {description}...")
    start_time = time.time()  # Record start time
    try:
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()
        end_time = time.time()  # Record end time
        duration = end_time - start_time # Calculate duration
        duration_formatted = "{:.2f}".format(duration) # Format duration to 2 decimal places

        if process.returncode != 0:
            print(f"{CROSS_MARK} Command failed: {description} (Took {duration_formatted} seconds)")
            print(f"   Command: {command}")
            print(f"   Error Output:\n{stderr.decode()}")
            return False
        else:
            update_progress(step_progress, total_step_commands) # Use updated progress function
            print(f"{CHECK_MARK} Command {step_progress}/{total_step_commands} completed: {description} (Took {duration_formatted} seconds)")
            return True

    except Exception as e:
        end_time = time.time()  # Record end time even if exception occurs
        duration = end_time - start_time # Calculate duration
        duration_formatted = "{:.2f}".format(duration) # Format duration
        print(f"{CROSS_MARK} A general error occurred during command: {description} (Took {duration_formatted} seconds)")
        print(f"   Error: {e}")
        return False

# --- Step 1: OS-Level Installations ---
os_install_commands = [
    ("sudo apt-get update", "Updating software package lists (Takes around 14.14 seconds)"),
    ("apt-get install -y libxcb-cursor0", "Install cursor display library (libxcb-cursor0 - for Calibre) (Takes around 12.99 seconds)"),
    ("sudo -v && wget -nv -O- https://download.calibre-ebook.com/linux-installer.sh | sudo sh /dev/stdin", "Download & install Calibre ebook manager(Takes around 29.40 seconds)"),
    ("apt-get install -y libegl1", "Install graphics library (libegl1) (Takes around 4.12 seconds)"),
    ("apt-get install -y libopengl0", "Install graphics library (libopengl0) (Takes around 3.07 seconds)"),
    ("apt-get install -y ffmpeg", "Install ffmpeg (audio/video conversion) (Takes around 3.52 seconds)"),
    ("apt-get install -y espeak-ng", "Install espeak-ng (espeak-ng tts engine) (Takes around 3.52 seconds)"),
    ("apt-get install -y mecab", "Install mecab (Japanese text analysis) (Takes around 14.51 seconds)"),
    ("apt-get install -y libmecab-dev", "Install mecab development files (Takes around 5.72 seconds)"),
    ("apt-get install -y mecab-ipadic-utf8", "Install mecab Japanese dictionary (UTF-8) (Takes around 9.58 seconds)"),
    ("apt-get install -y nodejs", "Install nodejs (Javascript runtime) (Takes around 7.03 seconds)"),
    ("pip install mecab-python3 unidic-lite", "Install mecab-python3 (Takes around 19.80 seconds)"),
    ("pip install unidic-lite", "Install lightweight unidic dictionary (Takes around 2.15 seconds)"),
    ("pip install unidic", "Install full Unidic dictionary (Takes around 4.00 seconds)"),
    ("python -m unidic download", "Download Unidic dictionary data (Takes around 29.05 seconds)"),
]
os_install_completed = False

# --- Step 2: Git Clone & Requirements ---
git_requirements_commands = [
    ("git clone https://github.com/DrewThomasson/ebook2audiobook.git", "Git clone Ebook2audiobook"),
#    ("pip install -r /content/ebook2audiobook/requirements.txt", "Install Python requirements (Takes around 202.4 seconds)"),
    ("pip install -r /kaggle/working/ebook2audiobook/requirements.txt", "Install Python requirements (Takes around 202.4 seconds)"), # modified path for kaggle
]
git_requirements_completed = False

# --- Step 3: Run App ---
run_app_completed = False


# --- Step 1 Execution ---
if os_install_completed:
    print("\nStep 1: OS-Level Installations - Already completed, skipping.")
else:
    print("\n--- Step 1: OS-Level Installations ---")
    print("This step installs essential software on the Colab system. (Takes around 3-4 minutes)")
    total_steps_step1 = len(os_install_commands)
    display_loading_bar(total_steps_step1) # Start enhanced loading indicator

    step1_success = True
    for i, (command, description) in enumerate(os_install_commands):
        if not run_command_with_log(command, description, i + 1, total_steps_step1):
            step1_success = False
            break

    if step1_success:
        os_install_completed = True
        print(f"\n{CHECK_MARK} Step 1: OS-Level Installations - Completed Successfully!")
    else:
        print(f"\n{CROSS_MARK} Step 1: OS-Level Installations - Failed. See error messages above.")


# --- Step 2 Execution ---
if git_requirements_completed:
    print("\nStep 2: Git Clone & Requirements - Already completed, skipping.")
else:
    print("\n--- Step 2: Git Clone & Requirements ---")
    print("This step downloads program files and installs Python components. (Takes around 3-4 minutes)")
    total_steps_step2 = len(git_requirements_commands)
    display_loading_bar(total_steps_step2) # Start enhanced loading indicator

    step2_success = True
    for i, (command, description) in enumerate(git_requirements_commands):
        if not run_command_with_log(command, description, i + 1, total_steps_step2):
            step2_success = False
            break

    if step2_success:
        git_requirements_completed = True
        print(f"\n{CHECK_MARK} Step 2: Git Clone & Requirements - Completed Successfully!")
    else:
        print(f"\n{CROSS_MARK} Step 2: Git Clone & Requirements - Failed. See error messages above.")


def modify_python_version(filepath, new_version=(3, 11)):
    """
    Modifies the min_python_version in a Python file.
    (Function definition - same as before)
    """
    try:
        with open(filepath, 'r') as f:
            file_content = f.read()
        pattern = r"min_python_version = \(\s*(\d+)\s*,\s*(\d+)\s*\)"
        def replacement(match):
            return f"min_python_version = ({new_version[0]}, {new_version[1]})"
        new_content = re.sub(pattern, replacement, file_content)
        if new_content != file_content:
            with open(filepath, 'w') as f:
                f.write(new_content)
            print(f"Successfully changed min_python_version in {filepath} to {new_version}")
        else:
            print(f"min_python_version in {filepath} was already {new_version} or not found.")
    except FileNotFoundError:
        print(f"Error: File not found: {filepath}")
    except Exception as e:
        print(f"An error occurred: {e}")

def fix_functions_py(filepath):
    """
    Scans functions.py and fixes the incorrect f-string issue if found.
    """
    # Define the incorrect pattern (allowing indentation and triple quotes)
    incorrect_pattern = r"msg\s*=\s*f'Voice file \{re\.sub\(r'_\(24000\|16000\)\\.wav\$', '', selected_name\)\} deleted!'"

    # Correct replacement (keeping indentation)
    correct_replacement = """                        cleaned_name = re.sub(r'_(24000|16000)\\.wav$', '', selected_name)
                        msg = f"Voice file {cleaned_name} deleted!" """

    try:
        # Read the file
        with open(filepath, "r") as file:
            lines = file.readlines()

        # Check and replace the incorrect line
        modified_lines = []
        fixed = False

        for line in lines:
            if re.search(incorrect_pattern, line.strip()):  # Use regex search instead of exact match
                print("🔍 Found incorrect line in functions.py. Fixing it now...")
                modified_lines.append(correct_replacement + "\n")  # Add the corrected version
                fixed = True
            else:
                modified_lines.append(line)  # Keep other lines unchanged

        # Only overwrite the file if a fix was applied
        if fixed:
            with open(filepath, "w") as file:
                file.writelines(modified_lines)
            print("✅ Fix applied successfully!")
        else:
            print("✅ No fix needed. The exact error line was not found.")

    except FileNotFoundError:
        print(f"❌ Error: The file {filepath} was not found.")
    except Exception as e:
        print(f"❌ An error occurred: {e}")

# Modify conf.py AFTER git clone (Step 2 ensures clone is done)
# filepath = "/content/ebook2audiobook/lib/conf.py" 
filepath = "/kaggle/working/ebook2audiobook/lib/conf.py" # kaggle file path change
modify_python_version(filepath)

# Call the function to fix functions.py
# filepath = "/content/ebook2audiobook/lib/functions.py"
filepath = "/kaggle/working/ebook2audiobook/lib/functions.py" # kaggle file path change
fix_functions_py(filepath)


# --- Step 3 Execution ---
if run_app_completed:
    print("\nStep 3: Run App - Already run (or skipped).")
else:
    print("\n--- Step 3: Run App ---")
    print("This step starts the ebook2audiobook web interface.")
    print("Starting ebook2audiobook...")
    try:
        !cd ebook2audiobook && python app.py --script_mode full_docker --share
#        !cd ebook2audiobook && python app.py --script_mode full_docker #fix gradio block
        run_app_completed = True
    except Exception as e:
        print(f"{CROSS_MARK} Error starting app.py: {e}")
    else:
        print(f"\n{CHECK_MARK} Step 3: Run App - Started! Check the output above for the Gradio link.")


print("\n--- All Steps Initiated (or Skipped if Already Run) ---")
print("Check the output above for details on each step.")